In [1]:
import pandas as pd
import os
import yaml
#!pip install xlrd==1.2.0

In [2]:
yml_path = os.path.join(os.getcwd(),'streetcar_data_preprocessing_config.yml')
print(yml_path)

/home/nishant/AI workspace/github/Deep Learning for Structured Data/notebooks/streetcar_data_preprocessing_config.yml


In [3]:
try:
    with open(yml_path, 'r') as f:
        config = yaml.safe_load(f)
except Exception as e:
    print('Error in reading config file')    

In [4]:
load_from_scrath = config['general']['load_from_scrath']
save_transformed_dataframe = config['general']['save_transformed_dataframe']
remove_bad_values = config['general']['remove_bad_values']
pickled_input_dataframe = config['full_names']['pickled_input_dataframe']
pickled_output_dataframe = config['full_names']['pickled_output_dataframe']

In [5]:
def get_path():
    rawpath = os.getcwd()
    path = os.path.abspath(os.path.join(rawpath, '..', 'data'))
    return path

In [7]:
path = get_path()
file_name = 'ttc-streetcar-delay-data-2014.xlsx'
data_path = os.path.join(path, file_name)

xlsf = pd.ExcelFile(data_path)
df = pd.read_excel(data_path, sheet_name = xlsf.sheet_names[0])

for sheet_name in xlsf.sheet_names[1:]:
        data = pd.read_excel(data_path, sheet_name = sheet_name)
        #print(data.head(5))
        df = df.append(data)

In [8]:
def fix_anomalous_columns(df):
    df['Min Delay'].fillna(df['Delay'], inplace=True)
    df['Min Gap'].fillna(df['Gap'], inplace=True)
    del df['Delay']
    del df['Gap']
    del df['Incident ID']
    return(df)

In [14]:
def load_xls(path, files_xls, firstfile, firstsheet, df):
    '''
    load all the tabs of all the XLS files in a list of XLS files, minus
    tab that has seeded dataframe
    Parameters:
    path: directory containing the XLS files
    files_xls: list of XLS files
    firstfile: file whose first tab has been preloaded
    firstsheet: first tab of the file that has been preloaded
    df: Pandas dataframe that has been preloaded with the first tab
    of the first XLS file and is loaded with all the data
    when the function returns

    Returns:
    df: updated dataframe
    '''

    for f in files_xls:
        print("filename",f)
        xlsf = pd.ExcelFile(path+f)
        for sheet_name in xlsf.sheet_names:
            print("sheet_name",sheet_name)
            if (f != firstfile) or (sheet_name != firstsheet):
                print("sheet_name in loop",sheet_name)
                data = pd.read_excel(path+f,sheetname=sheet_name)
                df = df.append(data)
        return (df)

In [17]:
def reloader(path,picklename):

    files_xls = get_xls_list(path)
    print("list of xls",files_xls)

    dfnew = pd.read_excel(path+files_xls[0])
    xlsf = pd.ExcelFile(path+files_xls[0])
    dflatest = load_xls(path,files_xls,files_xls[0],xlsf.sheet_names[0], dfnew)
    dflatest.to_pickle(os.path.join(path,picklename))
    return(dflatest)

## Load data from url and store it in pickle format

In [20]:
url='https://gist.githubusercontent.com/curran/a08a1080b88344b0c8a7/raw/d546eaee765268bf2f487608c537c05e22e4b221/iris.csv'

In [21]:
iris_df = pd.read_csv(url)
iris_df.head()

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


In [25]:
file_name = "iris_dataframe.pkl"
path = get_path()

In [26]:
iris_df.to_pickle(os.path.join(path, file_name))

## Load pickle data